In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, EfficientNetB0, resnet50, efficientnet
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------------
# 1. Load the EuroSAT Dataset
# ---------------------------------------------------
# Create train/validation/test split
(dataset_train, dataset_test), info = tfds.load(
    'eurosat/rgb',
    split=['train[:90%]', 'train[90%:]'],
    as_supervised=True,
    with_info=True
)

# Number of classes in EuroSAT
num_classes = info.features['label'].num_classes

# ---------------------------------------------------
# 2. Preprocess Data for ResNet and EfficientNet
# ---------------------------------------------------
IMG_SIZE = 224
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_resnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = resnet50.preprocess_input(image)
    return image, tf.one_hot(label, num_classes)

def preprocess_efficientnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = efficientnet.preprocess_input(image)
    return image, tf.one_hot(label, num_classes)

# Create tf.data pipelines
train_ds_resnet = (dataset_train
                   .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                   .batch(batch_size)
                   .prefetch(AUTOTUNE))

test_ds_resnet = (dataset_test
                  .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                  .batch(batch_size)
                  .prefetch(AUTOTUNE))

train_ds_eff = (dataset_train
                .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
                .batch(batch_size)
                .prefetch(AUTOTUNE))

test_ds_eff = (dataset_test
               .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
               .batch(batch_size)
               .prefetch(AUTOTUNE))

# ---------------------------------------------------
# 3. Define ResNet50 Model
# ---------------------------------------------------
def create_resnet(input_shape, num_classes):
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = True

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="ResNet")
    return model

# ---------------------------------------------------
# 4. Define EfficientNetB0 Model
# ---------------------------------------------------
def create_efficientnet(input_shape, num_classes):
    base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = True

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="EfficientNet")
    return model

# ---------------------------------------------------
# 5. Compile and Train the Models
# ---------------------------------------------------
# ResNet50 Model
resnet_model = create_resnet((IMG_SIZE, IMG_SIZE, 3), num_classes)
resnet_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning ResNet50 on EuroSAT ---")
history_resnet = resnet_model.fit(train_ds_resnet, epochs=3, validation_data=test_ds_resnet, verbose=1)

# EfficientNetB0 Model
eff_model = create_efficientnet((IMG_SIZE, IMG_SIZE, 3), num_classes)
eff_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning EfficientNetB0 on EuroSAT ---")
history_eff = eff_model.fit(train_ds_eff, epochs=3, validation_data=test_ds_eff, verbose=1)

# Evaluate ResNet50
loss_resnet, acc_resnet = resnet_model.evaluate(test_ds_resnet, verbose=0)
print(f"ResNet50 Final Accuracy: {acc_resnet:.4f}")

# Evaluate EfficientNetB0
loss_eff, acc_eff = eff_model.evaluate(test_ds_eff, verbose=0)
print(f"EfficientNetB0 Final Accuracy: {acc_eff:.4f}")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

760/760 ━━━━━━━━━━━━━━━━━━━━ 326s 342ms/step - accuracy: 0.8741 - loss: 0.3880 - val_accuracy: 0.9630 - val_loss: 0.1215
Epoch 2/3
760/760 ━━━━━━━━━━━━━━━━━━━━ 236s 311ms/step - accuracy: 0.9771 - loss: 0.0761 - val_accuracy: 0.9430 - val_loss: 0.1704
Epoch 3/3
760/760 ━━━━━━━━━━━━━━━━━━━━ 237s 312ms/step - accuracy: 0.9847 - loss: 0.0476 - val_accuracy: 0.9526 - val_loss: 0.1650
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

--- Fine-Tuning EfficientNetB0 on EuroSAT ---
Epoch 1/3
760/760 ━━━━━━━━━━━━━━━━━━━━ 242s 209ms/step - accuracy: 0.8032 - loss: 0.6831 - val_accuracy: 0.9737 - val_loss: 0.0771
Epoch 2/3
760/760 ━━━━━━━━━━━━━━━━━━━━ 106s 140ms/step - accuracy: 0.9726 - loss: 0.0901 - val_accuracy: 0.9793 - val_loss: 0.0567
Epoch 3/3
760/760 ━━━━━━━━━━━━━━━━━━━━ 105s 138ms/step - accuracy: 0.9876 - loss: 0.0406 - val_accuracy: 0.9826 - val_loss: 0.0475
ResNet50 Final Accuracy: 0.9526
EfficientNetB0 Final Accuracy: 0.9826
